I have the ditribution file for each source and target pair
How to merge multiple source and target probabilities?

In [1]:
import operator
import functools
from collections import defaultdict
import csv
from collections import defaultdict, Counter
import math
import numpy as np


from scripts import build_rules_utils as BRU
from scripts import utils
%cd /afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON/

/afs/crc.nd.edu/user/m/msaebi/Public/Code-SF-HON


In [2]:
Count = {}
Rules = defaultdict(dict)
Distribution = defaultdict(dict)
final_dist=defaultdict(dict)
SourceToExtSource = {}
Verbose = True

def Initialize():
    Count = {}
    Rules = defaultdict(dict)
    Distribution = defaultdict(dict)
    SourceToExtSource = {}

def ExtractRules(Trajectory, MaxOrder, MinSupport):
#    Initialize()
    Build_Observations_Distributions(Trajectory)
    Aggragate_Probs(Distribution,MinSupport)
    GenerateAllRules(MaxOrder)
    return Rules

#*******************************************
def Build_Observations_Distributions(Trajectory):
#build distribution based on observation file. append secveral probabilities
#cprresponding to the same source & target
    
    for record in Trajectory: #record=[movement, prob]
        trajectory = record[0]     #Modified for no ship data record[1] 
        Target = trajectory[-1]
        Source = tuple(trajectory[:-1])
        prob=float(record[1])

        if Source in Distribution:
            if Target not in Distribution[Source]:
                Distribution[Source].update({str(Target):[prob]})
            if Target in Distribution[Source]:
                Distribution[Source][Target].append(prob)                
        else:
            Distribution[Source][Target]=[prob]            
    return(Distribution)


def Aggragate_Probs(Distribution,MinSupport):
#aggeregate all probabilities to each other
    a=[]
    for source,value in Distribution.items():#value={target:prob}
        for target,val in value.items(): #key=target, val=prob
            if (len(val)>1):
                #a[:] = [1-x for x in val] #subtract all elemnts of the list from 1
                #final_prob=1-functools.reduce(operator.mul, a, 1) #multiply them by each other and subtract the result form 1
                final_prob=np.mean(val)
                if (final_prob>MinSupport):
                    final_dist[source][target]=float(final_prob)
            else:
                if (float(val[0])>MinSupport):
                    final_dist[source][target]=float(val[0])
    return(final_dist)

#*******************************************
def GenerateAllRules(MaxOrder):
    print('building cache')
    BuildSourceToExtSource() # to speed up lookups
    print('generating rules')
    #VPrint(len([x for x in final_dist if len(x) == 1]))
    LoopCounter = 0
    for Source in final_dist.copy():
        if len(Source) == 1:
            AddToRules(Source)
            ExtendRule(Source, Source, 1, MaxOrder)
            LoopCounter += 1
    return Rules

def ExtendRule(Valid, Curr, order, MaxOrder):
    if order >= MaxOrder:
        AddToRules(Valid)
    else:
        Distr = final_dist[Valid]
        NewOrder = order + 1
        Extended = ExtendSource(Curr, NewOrder)
        if len(Extended) == 0:
            AddToRules(Valid)
        else:
            for ExtSource in Extended:
                ExtDistr = final_dist[ExtSource] # Pseudocode in Algorithm 1 has a typo here
                if BRU.KLD(ExtDistr, Distr) > KLDThreshold(NewOrder, ExtSource):
                    # higher-order dependencies exist for order NewOrder
                    # keep comparing probability distribution of higher orders with current order
                    ExtendRule(ExtSource, ExtSource, NewOrder, MaxOrder)
                else:
                    # higher-order dependencies do not exist for current order
                    # keep comparing probability distribution of higher orders with known order
                    ExtendRule(Valid, ExtSource, NewOrder, MaxOrder)

def AddToRules(Source):
    if len(Source) > 0:
        ## To output frequencies instead of probabilities, change "Distribution" to "Count"
        ## and filter out zero values
        Rules[Source] = final_dist[Source]
        PrevSource = Source[:-1]
        AddToRules(PrevSource)

###########################################
# Auxiliary functions
###########################################

def ExtractSubSequences(trajectory, order):
    SubSequence = []
    for starting in range(len(trajectory) - order + 1):
        SubSequence.append(tuple(trajectory[starting:starting + order]))
    return SubSequence

def IncreaseCounter(Source, Target):
    if not Source in Count:
        Count[Source] = Counter()
    Count[Source][Target]+=1
    return Count

def ProdProb(Source, Target,Distribution):
    if not Source in Count:
        Count[Source] = Counter()
    Count[Source][Target] += 1
    return Count
#*******************************************
def ExtendSourceSlow(Curr, NewOrder):
    Extended = []
    for CandidateSource in final_dist:
        if len(CandidateSource) == NewOrder and CandidateSource[-len(Curr):] == Curr:
            Extended.append(CandidateSource)
    return Extended

def ExtendSource(Curr, NewOrder):
    if Curr in SourceToExtSource:
        if NewOrder in SourceToExtSource[Curr]:
            return SourceToExtSource[Curr][NewOrder]
    return []

## creating a cache for fast lookup
def BuildSourceToExtSource():
    for source in final_dist:
        if len(source) > 1:
            NewOrder = len(source)
            for starting in range(1, len(source)):
                curr = source[starting:]
                if not curr in SourceToExtSource:
                    SourceToExtSource[curr] = {}
                if not NewOrder in SourceToExtSource[curr]:
                    SourceToExtSource[curr][NewOrder] = set()
                SourceToExtSource[curr][NewOrder].add(source)
def KLDThreshold(NewOrder, ExtSource):
    #print("NewOrder",NewOrder)
    #print("ExtSource",ExtSource)
    return NewOrder / math.exp(1 + (sum(i for i in final_dist[ExtSource].values()))) # typo in Pseudocode in Algorithm 1



In [3]:
from collections import defaultdict, Counter

Graph = defaultdict(dict)
Verbose = True

def Initialize():
    global Graph
    Graph = defaultdict(dict)


def BuildNetwork(Rules):
    print('Building network')
    Initialize()
    SortedSource = sorted(Rules, key=lambda x: len(x))
    for source in SortedSource:
        for target in Rules[source]:
            Graph[source][(target,)] = Rules[source][target]
            # following operations are destructive to Rules
            if len(source) > 1:
                Rewire(source, (target,))
    RewireTails()
    return Graph

def Rewire(source, target):
    #print(source, target)
    PrevSource = source[:-1]
    PrevTarget = (source[-1],)
    if not PrevSource in Graph or not source in Graph[PrevSource]:
        try:
            Graph[PrevSource][source] = Graph[PrevSource][PrevTarget]
            del(Graph[PrevSource][PrevTarget])

        except:
            pass
def RewireTails():
    ToAdd = []
    ToRemove = []
    for source in Graph:
        for target in Graph[source]:
            if len(target) == 1:
                NewTarget = source + target
                while len(NewTarget) > 1:
                    if NewTarget in Graph:
                        ToAdd.append((source, NewTarget, Graph[source][target]))
                        ToRemove.append((source, target))
                        break
                    else:
                        NewTarget = NewTarget[1:]
    for (source, target, weight) in ToAdd:
        Graph[source][target] = weight
    for (source, target) in ToRemove:
        del(Graph[source][target])

In [4]:
year='1997_2018'
year='2018'
y=year+'/'
MaxOrder = 16
MinSupport = 1e-20

#Input:
port_data='data/Places_allportdata_mergedSept2017.csv'
#eco='noEco_';paul='';env='noEnv_'
r='r0_'
r=''
env='env_' ;eco='Eco_' ;paul=''
#eco='sameEco_';paul='zh/'+r+'/';env='env_'
InputForHON_Ballast='data/'+y+paul+'l_InputForHON_Ballast_' +env+eco+ str(MaxOrder)+'.csv'
InputForHON_Fouling='data/'+y+paul+'l_InputForHON_Fouling_' +env+eco+ str(MaxOrder)+'.csv'


#output:
OutputRulesFile_Ballast='data/'+y+paul+r+'Rules_Ballast_' +env+eco+ str(year)+'_'+str(MaxOrder)+'.csv'
OutputNetworkFile_Ballast='data/'+y+paul+r+'HONet_Ballast_' +env+eco+ str(year)+'_'+str(MaxOrder)+'.csv'

OutputRulesFile_Fouling='data/'+y+paul+'/l_Rules_Fouling_' +env+eco+ str(year)+'_'+str(MaxOrder)+'.csv'
OutputNetworkFile_Fouling='data/'+y+paul+'/l_HONet_Fouling_' +env+eco+ str(year)+'_'+str(MaxOrder)+'.csv'


In [5]:
#Choose biofouling or balast
#InputFileName , OutputRulesFile , OutputNetworkFile= InputForHON_Fouling , OutputRulesFile_Fouling , OutputNetworkFile_Fouling
InputFileName ,OutputRulesFile, OutputNetworkFile= InputForHON_Ballast , OutputRulesFile_Ballast, OutputNetworkFile_Ballast

LastStepsHoldOutForTesting = 1
MinimumLengthForTraining = 1
InputFileDeliminator = ' '

ports = utils.GetPortData(port_data, 'ID', ',')
RawTrajectories = BRU.ReadSequentialData(InputFileName,InputFileDeliminator, MinimumLengthForTraining, LastStepsHoldOutForTesting)
TrainingTrajectory, TestingTrajectory = BRU.BuildTrainingAndTesting(RawTrajectories,LastStepsHoldOutForTesting)

print("RAW Trajectory");BRU.VPrint(len(RawTrajectories))
print("Training Trajectory");BRU.VPrint(len(TrainingTrajectory))
print("Testing Trajectory");BRU.VPrint(len(TestingTrajectory))

Reading raw sequential data
Building training and testing
RAW Trajectory
17575305
Training Trajectory
17575305
Testing Trajectory
17575305


In [6]:
Build_Observations_Distributions(RawTrajectories)
Aggragate_Probs(Distribution,MinSupport)#eliminated 13.42 % of the paths(probs) 5935 of 6855. mean:0.00556928 Stdev 0.033
Rules= GenerateAllRules(MaxOrder)
Network = BuildNetwork(Rules)
BRU.DumpRules(Rules, OutputRulesFile)
BRU.DumpNetwork(Network, OutputNetworkFile)

building cache
generating rules
Building network
Dumping rules to file
Dumping network to file


### Basic network stats

In [13]:
import networkx as nx
import pandas as pd
order=16
r='r0'
eco='sameEco_';paul='zh/'+r+'/';env='env_'
HONet_translated_b='data/'+y+paul+r+'_trans_HONet_Ballast_'+ eco+env+year+'_'+str(order)+'.net'
HONet_translated_f='data/'+y+paul+r+'_trans_HONet_Fouling_'+ eco+env+year+'_'+str(order)+'.net'



In [14]:
f = pd.read_csv(OutputNetworkFile_Ballast, sep=',')
f = pd.read_csv(HONet_translated_b, sep=' ')
f.columns = ['source', 'target','weight']
G = nx.from_pandas_edgelist(f,source='source', target='target',edge_attr='weight',create_using=nx.DiGraph())
print(nx.info(G))
#f.head()

Name: 
Type: DiGraph
Number of nodes: 14674
Number of edges: 139474
Average in degree:   9.5048
Average out degree:   9.5048
